In [90]:
import numpy as np
import pandas as pd
import umap
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import HDBSCAN
import seaborn as sns
import matplotlib.pyplot as plt
#from .autonotebook import tqdm as notebook_tqdm

data = pd.read_csv(r'C:\Users\jdorv\Coding Fun\Bank Innovation\wrds_bank_data_MERGED.csv')


In [88]:
def merge_split_columns(df):
    """
    Merge RCFD/RCON columns that split in 2011.
    RCFD (consolidated) has data pre-2011, RCON (domestic) has data post-2011.
    Strategy: Create merged columns using RCFD when available, fill with RCON otherwise.
    """
    print(f"\n{'='*80}")
    print("MERGING SPLIT TIME CODES (2011 Transition)")
    print(f"{'='*80}")
    
    # Define split pairs: (rcfd_col, rcon_col, new_merged_name)
    split_pairs = [
        ('rcfd2_rcfd2170', 'rcon2_rcon2170', 'total_assets'),
        ('rcfd2_rcfd2122', 'rcon2_rcon2122', 'total_loans'),
        ('rcfd2_rcfd3210', 'rcon2_rcon3210', 'total_equity'),
        ('rcfd1_rcfd3123', 'rcon1_rcon3123', 'allowance_loan_losses'),
        ('rcfd1_rcfd1590', 'rcon1_rcon1590', 'agricultural_loans'),
        ('rcfd1_rcfd1754', 'rcon1_rcon1754', 'htm_securities'),
        ('rcfd1_rcfd1773', 'rcon1_rcon1773', 'afs_securities'),
        ('rcfd1_rcfd2150', 'rcon2_rcon2150', 'goodwill'),
        ('rcfd1_rcfd0081', 'rcon2_rcon0081', 'cash_items_process'),
        ('rcfd2_rcfd1420', 'rcon2_rcon1420', 'farmland_loans'),
        ('rcfd2_rcfd1460', 'rcon2_rcon1460', 'multifamily_loans'),
    ]
    
    merged_count = 0
    for rcfd_col, rcon_col, new_name in split_pairs:
        # Check if both columns exist
        rcfd_exists = rcfd_col in df.columns
        rcon_exists = rcon_col in df.columns
        
        if rcfd_exists and rcon_exists:
            # Merge: use RCFD, fill missing with RCON
            df[new_name] = df[rcfd_col].fillna(df[rcon_col])
            
            # Count how many values came from each source
            rcfd_count = df[rcfd_col].notna().sum()
            rcon_count = df[rcon_col].notna().sum()
            merged_total = df[new_name].notna().sum()
            
            print(f"✓ {new_name:25s} | RCFD: {rcfd_count:>6,} | RCON: {rcon_count:>6,} | Total: {merged_total:>6,}")
            
            # Drop original columns to avoid confusion
            df = df.drop(columns=[rcfd_col, rcon_col])
            merged_count += 1
            
        elif rcfd_exists and not rcon_exists:
            # Only RCFD exists, just rename it
            df[new_name] = df[rcfd_col]
            df = df.drop(columns=[rcfd_col])
            print(f"⚠️  {new_name:25s} | Only RCFD exists, renamed")
            merged_count += 1
            
        elif rcon_exists and not rcfd_exists:
            # Only RCON exists, just rename it
            df[new_name] = df[rcon_col]
            df = df.drop(columns=[rcon_col])
            print(f"⚠️  {new_name:25s} | Only RCON exists, renamed")
            merged_count += 1
            
        else:
            print(f"⚠️  {new_name:25s} | Neither column found, skipping")
    
    print(f"\n✓ Successfully merged/renamed {merged_count} split column pairs")
    return df


def calculate_ratios(df):
    """Calculate ratios using merged columns"""
    print(f"\n{'='*80}")
    print("CALCULATING FINANCIAL RATIOS (USING MERGED COLUMNS)")
    print(f"{'='*80}")
    
    ratios = df.copy()
    
    def safe_divide(numerator, denominator):
        result = numerator / denominator
        result = result.replace([np.inf, -np.inf], np.nan)
        return result
    
    # === CORE INNOVATION METRICS (3 ratios) ===
    ratios['tech_investment_ratio'] = safe_divide(
        df['riad4092'], df['total_assets']
    ) * 1000
    print("✓ Tech Investment Ratio")
    
    ratios['nib_deposit_ratio'] = safe_divide(
        df['rcon2_rcon6631'], df['rcon2_rcon2200']
    ) * 100
    print("✓ NIB Deposit Ratio (Digital Banking Proxy)")
    
    ratios['service_charge_intensity'] = safe_divide(
        df['riad4080'], df['rcon2_rcon2200']
    ) * 1000
    print("✓ Service Charge Intensity")
    
    # === EFFICIENCY METRICS (2 ratios) ===
    ratios['efficiency_ratio'] = safe_divide(
        df['riad4093'], df['riad4074'] + df['riad4079']
    ) * 100
    print("✓ Efficiency Ratio")
    
    ratios['nonint_income_pct'] = safe_divide(
        df['riad4079'], df['riad4074'] + df['riad4079']
    ) * 100
    print("✓ Noninterest Income %")
    
    # === BALANCE SHEET (3 ratios) ===
    ratios['loans_to_assets'] = safe_divide(
        df['total_loans'], df['total_assets']
    ) * 100
    print("✓ Loans-to-Assets")
    
    ratios['equity_to_assets'] = safe_divide(
        df['total_equity'], df['total_assets']
    ) * 100
    print("✓ Equity-to-Assets")
    
    ratios['deposits_to_assets'] = safe_divide(
        df['rcon2_rcon2200'], df['total_assets']
    ) * 100
    print("✓ Deposits-to-Assets")
    
    # === PROFITABILITY (2 ratios) ===
    ratios['roa'] = safe_divide(
        df['riad4340'], df['total_assets']
    ) * 100
    print("✓ ROA")
    
    ratios['roe'] = safe_divide(
        df['riad4340'], df['total_equity']
    ) * 100
    print("✓ ROE")
    
    # === DEPOSIT MIX (1 ratio) ===
    ratios['nontrans_deposits_pct'] = safe_divide(
        df['rcon2_rcon2215'], df['rcon2_rcon2200']
    ) * 100
    print("✓ Nontransaction Deposits %")
    
    print(f"\n✓ Created 11 ratios using merged columns")
    return ratios


def prepare_clustering_features(df):
    """Select universal coverage features AND keep identifiers"""
    print(f"\n{'='*80}")
    print("PREPARING CLUSTERING FEATURES - UNIVERSAL COVERAGE ONLY")
    print(f"{'='*80}")
    
    # RIAD columns - Universal coverage (<2% missing)
    riad_universal = [
        'riad4010', 'riad4012', 'riad4020', 'riad4073', 'riad4074', 'riad4079',
        'riad4080', 'riad4092', 'riad4093', 'riad4107', 'riad4115', 'riad4135',
        'riad4150', 'riad4180', 'riad4217', 'riad4230', 'riad4266', 'riad4267',
        'riad4300', 'riad4301', 'riad4302', 'riad4313', 'riad4340', 'riad4356',
        'riad4415', 'riad4435', 'riad4436', 'riad4460', 'riad4470', 'riad4498',
        'riad4499', 'riad4507', 'riad4508', 'riad4518', 'riad4605', 'riad4608',
        'riad4628', 'riad4635', 'riad4638', 'riad4644', 'riad4769'
    ]
    
    # RCON columns - Universal coverage (<2% missing)
    # NOTE: Split columns have been merged, so we now use merged names
    rcon_universal = [
        'rcon2_rcon2200',  # Total deposits (no split)
        'rcon2_rcon2202',  # Transaction accounts (no split)
        'rcon2_rcon2215',  # Nontransaction accounts (no split)
        'rcon2_rcon6631',  # NIB deposits (no split)
        'rcon1_rcon1766',  # C&I loans (no split)
    ]
    
    # Merged columns (from split pairs)
    merged_columns = [
        'total_assets',
        'total_loans',
        'total_equity',
        'allowance_loan_losses',
        'agricultural_loans',
        'htm_securities',
        'afs_securities',
        'goodwill',
        'cash_items_process',
        'farmland_loans',
        'multifamily_loans',
    ]
    
    # Ratios - Successfully calculated (<2% missing)
    ratios_clean = [
        'tech_investment_ratio',
        'nib_deposit_ratio',
        'service_charge_intensity',
        'efficiency_ratio',
        'nonint_income_pct',
        'loans_to_assets',
        'equity_to_assets',
        'deposits_to_assets',
        'roa',
        'roe',
        'nontrans_deposits_pct'
    ]
    
    # Identifiers - MUST KEEP THESE
    identifiers = ['rssd9001', 'rssd9999', 'rssd9017', 'year', 'quarter']
    
    # Check which identifiers exist
    existing_identifiers = [col for col in identifiers if col in df.columns]
    if len(existing_identifiers) < len(identifiers):
        missing = [col for col in identifiers if col not in df.columns]
        print(f"\n⚠️  WARNING: Missing identifier columns: {missing}")
    
    # Combine all feature columns
    feature_cols = riad_universal + rcon_universal + merged_columns + ratios_clean
    
    # Check which features actually exist
    existing_features = [col for col in feature_cols if col in df.columns]
    missing_features = [col for col in feature_cols if col not in df.columns]
    
    print(f"\n📊 Feature selection:")
    print(f"  Expected: {len(feature_cols)} features")
    print(f"  Found: {len(existing_features)} features")
    if missing_features:
        print(f"  Missing: {len(missing_features)} features: {missing_features}")
    
    # Create dataset with BOTH identifiers AND features
    all_cols = existing_identifiers + existing_features
    df_subset = df[all_cols].copy()

    print(f"\n🔍 Missing value summary:")
    # Check missing values ONLY in feature columns
    features_only = df_subset[existing_features]
    total_missing = features_only.isna().sum().sum()
    total_cells = features_only.shape[0] * features_only.shape[1]
    missing_pct = (total_missing / total_cells) * 100
    print(f"  Total missing values in FEATURES: {total_missing:,} ({missing_pct:.2f}% of feature cells)")

    # Drop rows with ANY missing values IN FEATURE COLUMNS ONLY
    print(f"\n{'='*80}")
    print("REMOVING ROWS WITH MISSING VALUES (checking features only)")
    print(f"{'='*80}")

    before_rows = len(df_subset)
    # Get mask of complete rows based on features only
    complete_features_mask = features_only.notna().all(axis=1)
    df_clean = df_subset[complete_features_mask].copy()
    after_rows = len(df_clean)

    print(f"  Before: {before_rows:,} rows")
    print(f"  After:  {after_rows:,} rows")
    print(f"  Retained: {after_rows/before_rows*100:.1f}%")
    
    print(f"\n{'='*80}")
    print(f"✓ FINAL DATASET READY FOR CLUSTERING")
    print(f"{'='*80}")
    print(f"  Observations: {len(df_clean):,}")
    print(f"  Identifiers: {len(existing_identifiers)} columns - {existing_identifiers}")
    print(f"  Features: {len(existing_features)} columns")
    print(f"  Total columns: {len(df_clean.columns)}")
    print(f"  Missing values: {df_clean.isna().sum().sum()}")
    print(f"\n  Ready for: Z-score standardization → UMAP → HDBSCAN")
    
    # Return COMPLETE dataframe (identifiers + features)
    return df_clean, existing_features

def assign_sticky_bank_tiers(df, asset_col='total_assets', min_consecutive_quarters=3):
    """
    Assign bank tiers with stickiness - requires crossing threshold for 
    min_consecutive_quarters before tier changes.
    
    Parameters:
    -----------
    df : DataFrame with columns [rssd9017, year, quarter, asset_col]
    asset_col : str, name of the assets column
    min_consecutive_quarters : int, number of consecutive quarters needed to change tier
    
    Returns:
    --------
    df : DataFrame with new 'bank_tier' column
    """
    print(f"\n{'='*80}")
    print(f"ASSIGNING STICKY BANK TIERS ({min_consecutive_quarters} consecutive quarters)")
    print(f"{'='*80}")
    
    # Tier thresholds (in thousands)
    SMALL_THRESHOLD = 1_000_000
    MEDIUM_THRESHOLD = 10_000_000
    
    def get_raw_tier(assets):
        """Get tier based on current asset value"""
        if pd.isna(assets):
            return None
        elif assets < SMALL_THRESHOLD:
            return "Small"
        elif assets < MEDIUM_THRESHOLD:
            return "Medium"
        else:
            return "Large"
    
    # Sort by bank and time
    df = df.sort_values(['rssd9017', 'year', 'quarter']).copy()
    
    # Calculate raw tier for each observation
    df['raw_tier'] = df[asset_col].apply(get_raw_tier)
    
    # Initialize sticky tier column
    df['bank_tier'] = None
    
    # Process each bank separately
    tier_changes = 0
    total_banks = df['rssd9017'].nunique()
    
    for bank_id in df['rssd9017'].unique():
        bank_mask = df['rssd9017'] == bank_id
        bank_data = df.loc[bank_mask].copy()
        
        # Start with first observation's tier
        current_tier = bank_data['raw_tier'].iloc[0]
        df.loc[bank_mask, 'bank_tier'] = current_tier
        
        consecutive_count = 0
        potential_new_tier = None
        
        # Iterate through each quarter for this bank
        for idx in bank_data.index[1:]:
            raw_tier = df.loc[idx, 'raw_tier']
            
            # If raw tier suggests a change
            if raw_tier != current_tier:
                # If this is the same potential change as before, increment counter
                if raw_tier == potential_new_tier:
                    consecutive_count += 1
                else:
                    # New potential change, reset counter
                    potential_new_tier = raw_tier
                    consecutive_count = 1
                
                # If we've hit the threshold, make the change
                if consecutive_count >= min_consecutive_quarters:
                    current_tier = potential_new_tier
                    tier_changes += 1
                    consecutive_count = 0
                    potential_new_tier = None
            else:
                # Raw tier matches current tier, reset any pending change
                consecutive_count = 0
                potential_new_tier = None
            
            # Assign current sticky tier
            df.loc[idx, 'bank_tier'] = current_tier
    
    print(f"✓ Processed {total_banks:,} banks")
    print(f"✓ Total tier changes: {tier_changes:,}")
    print(f"\nTier distribution:")
    tier_counts = df.groupby('bank_tier')['rssd9017'].nunique()
    for tier in ['Small', 'Medium', 'Large']:
        if tier in tier_counts.index:
            count = tier_counts[tier]
            pct = (count / total_banks) * 100
            print(f"  {tier:8s}: {count:>6,} banks ({pct:>5.1f}%)")
    
    # Drop the temporary raw_tier column
    df = df.drop(columns=['raw_tier'])
    
    return df
def calculate_additional_innovation_ratios(df):
    """Calculate additional innovation and efficiency metrics"""
    
    ratios = df.copy()
    
    def safe_divide(numerator, denominator):
        result = numerator / denominator
        result = result.replace([np.inf, -np.inf], np.nan)
        return result
    
    # === ADDITIONAL INNOVATION METRICS ===
    
    # Digital revenue intensity (credit card fees relative to total revenue)
    ratios['digital_revenue_ratio'] = safe_divide(
        df['riad4415'],  # Credit card fees
        df['riad4074'] + df['riad4079']  # Total revenue
    ) * 100
    print("✓ Digital Revenue Ratio")
    
    # Non-branch revenue (noninterest income minus service charges)
    ratios['non_branch_revenue_pct'] = safe_divide(
        df['riad4079'] - df['riad4080'],  # Noninterest income minus service charges
        df['riad4074'] + df['riad4079']
    ) * 100
    print("✓ Non-Branch Revenue %")
    
    # Loan efficiency (interest income per dollar of loans)
    ratios['loan_yield'] = safe_divide(
        df['riad4107'],  # Interest income on loans
        df['total_loans']
    ) * 100
    print("✓ Loan Yield")
    
    # Securities intensity (investment in securities relative to assets)
    ratios['securities_to_assets'] = safe_divide(
        df['htm_securities'] + df['afs_securities'],
        df['total_assets']
    ) * 100
    print("✓ Securities to Assets")
    
    # Operating leverage (noninterest expense per salary dollar)
    ratios['expense_per_salary_dollar'] = safe_divide(
        df['riad4093'],  # Total noninterest expense
        df['riad4135']  # Salaries
    )
    print("✓ Expense per Salary Dollar")
    
    # Occupancy efficiency (occupancy expense relative to total assets)
    ratios['occupancy_intensity'] = safe_divide(
        df['riad4115'],  # Occupancy expense
        df['total_assets']
    ) * 1000
    print("✓ Occupancy Intensity")
    
    # Credit quality indicators
    ratios['chargeoff_rate'] = safe_divide(
        df['riad4635'],  # Total charge-offs
        df['total_loans']
    ) * 100
    print("✓ Charge-off Rate")
    
    ratios['provision_intensity'] = safe_divide(
        df['riad4230'],  # Provision for loan losses
        df['total_loans']
    ) * 100
    print("✓ Provision Intensity")
    
    # Capital efficiency
    ratios['asset_growth_capacity'] = safe_divide(
        df['total_equity'],
        df['total_assets']
    ) * 100
    print("✓ Asset Growth Capacity")
    
    print(f"\n✓ Created 9 additional innovation/efficiency ratios")
    return ratios

def calculate_innovation_change_scores(df, feature_list, min_years=10):
    """
    Calculate change in innovation metrics from first to last year for each bank.
    Each bank will appear ONCE in the output.
    
    Parameters:
    -----------
    df : DataFrame with bank-year observations
    feature_list : list of features to calculate changes for
    min_years : minimum number of years a bank must have data for (default 10)
    
    Returns:
    --------
    df_changes : DataFrame with one row per bank showing feature changes
    """
    print(f"\n{'='*80}")
    print("CALCULATING INNOVATION CHANGE SCORES (2010-2021)")
    print(f"{'='*80}")
    
    bank_changes = []
    banks_processed = 0
    banks_excluded = 0
    
    for bank_id in df['rssd9017'].unique():
        bank_data = df[df['rssd9017'] == bank_id].sort_values('year')
        
        # Require minimum years of data
        if len(bank_data) >= min_years:
            # Get first and last year data
            first_year = bank_data.iloc[0]
            last_year = bank_data.iloc[-1]
            
            changes = {}
            changes['rssd9017'] = bank_id
            changes['rssd9017_name'] = first_year['rssd9017'] if 'rssd9017' in bank_data.columns else bank_id
            changes['bank_tier'] = last_year['bank_tier']
            changes['first_year'] = first_year['year']
            changes['last_year'] = last_year['year']
            changes['years_observed'] = len(bank_data)
            
            # Calculate change for each feature
            for feat in feature_list:
                if feat in bank_data.columns:
                    changes[f'{feat}_change'] = last_year[feat] - first_year[feat]
                    changes[f'{feat}_first'] = first_year[feat]
                    changes[f'{feat}_last'] = last_year[feat]
            
            bank_changes.append(changes)
            banks_processed += 1
        else:
            banks_excluded += 1
    
    df_changes = pd.DataFrame(bank_changes)
    
    print(f"\n✓ Processed {banks_processed:,} banks")
    print(f"✗ Excluded {banks_excluded:,} banks (less than {min_years} years of data)")
    print(f"\nTier distribution:")
    tier_counts = df_changes['bank_tier'].value_counts()
    for tier in ['Small', 'Medium', 'Large']:
        if tier in tier_counts.index:
            print(f"  {tier:8s}: {tier_counts[tier]:>6,} banks")
    
    # Check for missing values in change scores
    change_cols = [col for col in df_changes.columns if col.endswith('_change')]
    missing_pct = (df_changes[change_cols].isna().sum().sum() / 
                   (len(df_changes) * len(change_cols))) * 100
    print(f"\nMissing values in change scores: {missing_pct:.2f}%")
    
    return df_changes

In [91]:
# === WORKFLOW WITH NEW RATIOS AND CHANGE SCORES ===

# Load and prepare data
df = data.copy()
df['report_date'] = pd.to_datetime(data['rssd9999'], errors='coerce')
df['year'] = df['report_date'].dt.year
df['quarter'] = df['report_date'].dt.quarter

# Step 1: Merge split columns
df = merge_split_columns(df)

# Step 2: Calculate original ratios
df_ratios = calculate_ratios(df)

# Step 3: Calculate additional innovation ratios
df_ratios = calculate_additional_innovation_ratios(df_ratios)

# Step 4: Prepare features for clustering
df_umap, feature_names = prepare_clustering_features(df_ratios)

# Step 5: Assign sticky bank tiers
df_umap = assign_sticky_bank_tiers(df_umap, asset_col='total_assets', min_consecutive_quarters=3)

# Step 6: Aggregate to bank-year level
bank_year_aggregated = df_umap.groupby(['rssd9017', 'year', 'bank_tier'])[feature_names].mean().reset_index()

print(f"\n✓ Bank-year aggregated: {len(bank_year_aggregated):,} observations")

# Step 7: Define innovation-only features (size-independent ratios)
innovation_only_features = [
    # Original ratios
    'tech_investment_ratio',
    'nib_deposit_ratio', 
    'service_charge_intensity',
    'efficiency_ratio',
    'nonint_income_pct',
    'loans_to_assets',
    'equity_to_assets',
    'deposits_to_assets',
    'roa',
    'roe',
    'nontrans_deposits_pct',
    
    # New ratios
    'digital_revenue_ratio',
    'non_branch_revenue_pct',
    'loan_yield',
    'securities_to_assets',
    'expense_per_salary_dollar',
    'occupancy_intensity',
    'chargeoff_rate',
    'provision_intensity',
    'asset_growth_capacity'
]

# Filter to only features that exist
innovation_features_available = [f for f in innovation_only_features if f in bank_year_aggregated.columns]
print(f"\n✓ Using {len(innovation_features_available)} innovation-only features for change scores")

# Step 8: Calculate change scores (each bank appears ONCE)
df_changes = calculate_innovation_change_scores(
    bank_year_aggregated, 
    innovation_features_available,
    min_years=9
)

# Step 9: Prepare change scores for clustering
change_feature_cols = [col for col in df_changes.columns if col.endswith('_change')]

print(f"\n✓ Ready to cluster on {len(change_feature_cols)} change features")
print(f"✓ Dataset: {len(df_changes):,} banks (each appears once)")


MERGING SPLIT TIME CODES (2011 Transition)
✓ total_assets              | RCFD:  4,201 | RCON: 292,714 | Total: 296,463
✓ total_loans               | RCFD:  4,201 | RCON: 296,463 | Total: 296,463
✓ total_equity              | RCFD:  4,201 | RCON: 292,262 | Total: 296,463
✓ allowance_loan_losses     | RCFD:  4,201 | RCON: 292,262 | Total: 296,463
✓ agricultural_loans        | RCFD:  4,201 | RCON: 296,463 | Total: 296,463
✓ htm_securities            | RCFD:  4,201 | RCON: 296,217 | Total: 296,463
✓ afs_securities            | RCFD:  4,201 | RCON: 296,217 | Total: 296,463
✓ goodwill                  | RCFD:  4,201 | RCON: 292,262 | Total: 296,463
✓ cash_items_process        | RCFD:  4,201 | RCON: 292,262 | Total: 296,463
✓ farmland_loans            | RCFD:  1,720 | RCON: 296,463 | Total: 296,463
✓ multifamily_loans         | RCFD:  1,720 | RCON: 296,463 | Total: 296,463

✓ Successfully merged/renamed 11 split column pairs

CALCULATING FINANCIAL RATIOS (USING MERGED COLUMNS)
✓ Tech Investm

In [92]:
# Cluster by tier using CHANGE SCORES
by_tier = df_changes.groupby("bank_tier")

reducer = umap.UMAP(n_neighbors=15, min_dist=0.1, metric='euclidean', random_state=42)
embedding = {}
clusters = {}

for tier, data in by_tier:
    print(f"\nProcessing {tier} banks: {len(data):,} observations")
    
    # Use only change score features
    scaled = StandardScaler().fit_transform(data[change_feature_cols])
    embedding[tier] = reducer.fit_transform(scaled)
    
    # Adjust HDBSCAN params based on tier size
    if tier == 'Large':
        min_cluster_size = 15  # Lower for small sample
        min_samples = 3
    elif tier == 'Medium':
        min_cluster_size = 30
        min_samples = 5
    else:  # Small
        min_cluster_size = 50
        min_samples = 10
    
    clusterer = HDBSCAN(
        min_cluster_size=min_cluster_size, 
        min_samples=min_samples,
        cluster_selection_method='eom'
    )
    clusters[tier] = clusterer.fit_predict(embedding[tier])
    
    n_clusters = len(set(clusters[tier])) - (1 if -1 in clusters[tier] else 0)
    n_noise = (clusters[tier] == -1).sum()
    
    print(f"  Clusters found: {n_clusters}")
    print(f"  Noise points: {n_noise:,} ({n_noise/len(data)*100:.1f}%)")

# Add results back to dataframe
df_changes['innovation_cluster'] = -1
df_changes['umap_1'] = np.nan
df_changes['umap_2'] = np.nan

for tier, data in by_tier:
    df_changes.loc[data.index, 'innovation_cluster'] = clusters[tier]
    df_changes.loc[data.index, 'umap_1'] = embedding[tier][:, 0]
    df_changes.loc[data.index, 'umap_2'] = embedding[tier][:, 1]

print(f"\n✓ Clustering complete!")


Processing Large banks: 71 observations
  Clusters found: 2
  Noise points: 16 (22.5%)

Processing Medium banks: 619 observations


c:\Users\jdorv\Coding Fun\Bank Innovation\Final-project\.venv\Lib\site-packages\umap\umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


  Clusters found: 5
  Noise points: 236 (38.1%)

Processing Small banks: 3,658 observations
  Clusters found: 3
  Noise points: 745 (20.4%)

✓ Clustering complete!


In [ ]:
def analyze_cluster_by_size(df, tier):
#"Check if clusters within a tier are just grouping by asset size"
print(f"\n{'='*80}")
print(f"CLUSTER vs ASSET SIZE ANALYSIS - {tier} Banks")
print(f"{'='*80}")

tier_data = df[df['bank_tier'] == tier].copy()

# Get clusters (excluding noise)
clusters = [c for c in tier_data['innovation_cluster'].unique() if c != -1]
clusters.sort()

print(f"\nAsset statistics by cluster:")
print(f"{'Cluster':<10} {'Count':>8} {'Mean Assets':>15} {'Median Assets':>15} {'Min Assets':>15} {'Max Assets':>15}")
print("-" * 80)

for cluster in clusters:
    cluster_data = tier_data[tier_data['innovation_cluster'] == cluster]
    assets = cluster_data['total_assets']
    
    print(f"{cluster:<10} {len(cluster_data):>8,} {assets.mean():>15,.0f} {assets.median():>15,.0f} {assets.min():>15,.0f} {assets.max():>15,.0f}")

# Handle noise separately if it exists
if -1 in tier_data['innovation_cluster'].values:
    noise_data = tier_data[tier_data['innovation_cluster'] == -1]
    assets = noise_data['total_assets']
    print(f"{'Noise':<10} {len(noise_data):>8,} {assets.mean():>15,.0f} {assets.median():>15,.0f} {assets.min():>15,.0f} {assets.max():>15,.0f}")

# Statistical test: ANOVA on assets across clusters
from scipy import stats
cluster_assets = [tier_data[tier_data['innovation_cluster'] == c]['total_assets'].values 
                    for c in clusters]

if len(clusters) > 1:
    f_stat, p_value = stats.f_oneway(*cluster_assets)
    print(f"\nANOVA test for asset size across clusters:")
    print(f"  F-statistic: {f_stat:.4f}")
    print(f"  p-value: {p_value:.6f}")
    
    if p_value < 0.001:
        print(f"  ⚠️  WARNING: Clusters have significantly different asset sizes (p < 0.001)")
        print(f"     This suggests clustering may be driven by size, not innovation")
    else:
        print(f"  ✓ Clusters have similar asset distributions (p >= 0.001)")

# Run for each tier
for tier in ['Large', 'Medium', 'Small']:
    analyze_cluster_by_size(bank_year_aggregated, tier)

IndentationError: expected an indented block after function definition on line 1 (1687162075.py, line 2)